# SpecMod Tutorial Notebook

This notebook is intended to guide the user on how to: 

1. preprocess data for input to specmod
2. calculate spectra for modeling
3. model earthquake spectra (although it may still be used outside of earthquake sources)

## Pre Processing Module Imports
For the calculation of spectra, SpecMod requires two ObsPy streams:
1. signal
2. noise

It requires the user to cut their waveforms for their specific needs e.g. 'n' second P-phase window and 'x' seconds of window of pre-signal noise. 

It is up to you to cut them, some things to consider are:

1. what phase is required - P or S?
2. what is the most appropriate channel (e.g. vertical (Z) for P, transverse (T) or east/west (E/W) for S)
3. do you trust your picks - are they real / theoretical - what are rough errors?
4. how many seconds of noise do you want - pre or post signal?
5. do I need to add station specific time shifts - is one station off consistently?
6. do I need to make my windows slightly larger to account for errors in location/timing/picking?

I have provided some basic tools, but you do not need to use them at all if you don't want to! It is only required that each trace in each stream has the following metadata (set *MANUALLY*) in their .stats dictionary:

    1. tr.stats['dep'] = depth (kilometers)
    2. tr.stats['olon'] = origin longitude (decimal degrees)
    3. tr.stats['olat'] = origin latitude (decimal degrees)
    4. tr.stats['olat'] = origin latitude (decimal degrees)
    4. tr.stats['slon'] = station longitude (decimal degrees)
    5. tr.stats['slat'] = station latitude (decimal degrees)
    6. tr.stats['selv'] = station elevation (meters)
    7. tr.stats['repi'] = epicentral distance (source-reciever) (kilometers)
    6. tr.stats['rhyp'] = hypocentral distance (source-reciever) (kilometers)

In [1]:
import os
os.chdir("../")
import specmod.utils as ut
import specmod.PreProcess as pre
from obspy import read, read_inventory, UTCDateTime
os.chdir("Tutorial")

In [2]:
# Define parent directories
pdata = "Data/2019-08-26T07:30:47.0"
pinv = "MetaData/"

In [3]:
#ut.read_pyrocko() 

In [4]:
# Space for custom classes / functions


In [5]:
# Earthquake origin information
olat, olon, odep, otime = 53.784, -2.967, 2.1, UTCDateTime("2019-08-26T07:49:24.2")

In [ ]:
# Read in our data (Vertical Channel)
st = read(os.path.join(pdata, '*HHZ*'), format='mseed')
# Read in the inventory (station XML file)
inv = read_inventory(os.path.join(pinv, "pnr_inventory.xml"), "stationxml")

In [ ]:
# set the distances for the stream (required for theoreticals)
pre.set_stream_distance(st, olat, olon, odep, otime, inventory=inv, dtype="mseed")

# set the picks
pre.set_picks_from_pyrocko(st, os.path.join(pdata, "2019-08-26T07:30:47.000000.picks"))

# remove the traces with no p-pick 
for tr in st:
    try:
        tr.stats['p_time']
    except KeyError:
        st.remove(tr)

In [ ]:
# pre-process
st.detrend("linear")
st.detrend("demean")
st.taper(0.05)
st.remove_response(inv)

In [ ]:
# quick sanity check
import matplotlib.pyplot as plt
for tr in st:
    plt.plot(tr.data, label=tr.id)
plt.legend()

In [ ]:
st[0].stats

In [ ]:
ut.plot_traces(st.copy(), plot_theoreticals=True, conv=1)

In [ ]:
sig = pre.get_signal(st, pre.cut_p, bf=0, raf=0.8)
noise = pre.get_noise_p(st, sig, bf=1, bshift=0)

In [ ]:
ut.plot_traces(st.copy(), plot_windows=True, conv=1, sig=sig, noise=noise)

# PART 2 - Process Spectra

Now the hard work is done, SpecMod will do the heavy lifting of spectral analysis for you.
This section will cover:
 1. creating spectra from the time series data you created earlier (ObsPy streams)
 2. saving the event spectra
 3. data manipulation (power spectra to amplitude, time integration/differentiation)
 5. visualisation

In [ ]:
# import the spectral module to calculate spectra from signal and noise windows
os.chdir("../")
import specmod.Spectral as sp
os.chdir("Tutorial")

In [ ]:
outpdir = "Spectra/"


In [ ]:
spectra = sp.Spectra.from_streams(sig, noise, quadratic=True, number_of_tapers=5)

The spectra are computed using a multi-taper approach (Prieto, G. A., R. L. Parker, F. L. Vernon. (2009))

In [ ]:
spectra.psd_to_amp() # change from PSD to AMP
spectra.quick_vis()

Note that the optimal signal bandwidth has been pre-calculated where signal-to-noise-ratio >= 3. *BEWARE FEATURE IS BUGGY SOMETIMES*

You can currently save the spectra out to a binary format called 'Pickle'.

In [ ]:
# save the spectra 
spectra.write_spectra(os.path.join(outpdir, "2019-08-26T07:30:47.0"), spectra, method='pickle')
os.listdir(outpdir) # list the directory for proof!

You MUST be careful when reading pickle files. Only read ones that you made or know who made them.

In [ ]:
spectra = sp.Spectra.read_spectra(os.path.join(outpdir, "2019-08-26T07:30:47.0"+".spec"), method='pickle', skip_warning=True)

You can integrate to displacement easily. Assuming you removed the response to ground velocity, integrate once.

In [ ]:
# you can integrate
spectra.inte()
spectra.quick_vis()

You can go back easily!

In [ ]:
spectra = sp.Spectra.read_spectra(os.path.join(outpdir, "2019-08-26T07:30:47.0"+".spec"), method='pickle', skip_warning=True)
spectra.quick_vis()

# PART 3 - Modeling Spectra

## The part you've been waiting for!

In [ ]:
# import the spectral module to calculate spectra from signal and noise windows
os.chdir("../")
import specmod.Fitting as fit
import specmod.Models as mod
os.chdir("Tutorial")

Spectral Fitting is a little more involved, right now only simple models can be fit:

$A(f) = \frac{{\Omega_0}} {(1+(f/f_c)^\gamma)^{\gamma n}}*\exp(-\pi f t^*)$, where the free parameters are $\Omega_0$, $f_c$ and $t^*$.

First, create a guess for those parameters for each spectrum. Conveniently, I have included a function to help you with this...


In [ ]:
guess = fit.FitSpectra.create_simple_guess(spectra)

This next step will initialse the fitting process.

In [ ]:
fits = fit.FitSpectra(spectra, mod.simple_model, guess)
fits.set_bounds('ts', min=0.0001) # you can set bounds for parameters (not sensible to have a ts < 0)

Then we will fit the spectra. For this we will use Powell's minimisation. 
The advantage of using the lmfit package is that it supports many search methods.
Check the docs for lmfit for more info!

In [ ]:
fits.fit_spectra(method='powell')

Visualise the fits.

In [ ]:
fits.quick_vis()

Then inspect the output!

In [ ]:
fits.table

Finally, save the output!

In [ ]:
fits.write_flatfile(os.path.join(outpdir, "FlatFiles", fits.spectra.event + ".csv"), fits)

In [ ]:
os.listdir(os.path.join(outpdir, "FlatFiles"))